### This notebook shows how to make a pandas DataFrame from a WarpScript GTS

In [1]:
%load_ext warpscript_cellmagic
%alias_magic w warpscript

Created `%%w` as an alias for `%%warpscript`.


In [2]:
import pandas as pd
import pickle as pkl

In [3]:
%%w -s s
NEWGTS 'randGTS' RENAME 1 10 <% h RAND RAND NaN RAND ADDVALUE %> FOR
NEWGTS 'nogeoTS' RENAME 2 11 <% h NaN NaN NaN RAND ADDVALUE %> FOR

Starting connection with 127.0.0.1:25333.
Creating a new WarpScript stack accessible under variable "s".
top: 	<GTS with 10 values>
1: 	<GTS with 10 values>



In [4]:
%%w -s s -o
<%
    # Check there is at least two params
    <% DEPTH 2 < %> <% 'Macro takes at least 2 arguments' MSGFAIL %> IFT
        
    # Check that top is a boolean indicating whether to use GTS classname or selector
    <% 1 PICK TYPEOF 'BOOLEAN' != %> <% 'Top of the stack must be a boolean' MSGFAIL %> IFT
    
    # Check that the top is a GTS
    <% 2 PICK TYPEOF 'GTS' != %> <% 'Below top of the stack must be a GTS' MSGFAIL %> IFT
    
    # Store the arguments
    'withSelector' STORE
    'gts' STORE
    
    # Make name
    # $gts NAME <% $withSelector %> <% $gts LABELS ->JSON + %> IFT
    $gts <% $withSelector %> <% TOSELECTOR %> <% NAME %> IFTE
    'name' STORE
    
    # macro: check not all NaN (for locations and elevations)
    <% UNIQUE DUP SIZE 1 == SWAP 0 GET ISNaN && %> 'isAllNaN' STORE
        
    # Return dict for pandas
    {
        # ticks
        'timestamps' $gts TICKS
        
        # locations
        $gts LOCATIONS 'lon' STORE 'lat' STORE
        <% $lat @isAllNaN ! %> <% $name '.lat' + $lat %> IFT
        <% $lon @isAllNaN ! %> <% $name '.lon' + $lon %> IFT
        
        # elevations
        $gts ELEVATIONS 'elev' STORE
        <% $elev @isAllNaN ! %> <% $name '.elev' + $elev %> IFT
        
        # values        
        $name $gts VALUES
    }
    ->PICKLE
%>
'GTStoPickledDict' STORE

Creating a new WarpScript stack accessible under variable "s".



In [5]:
%%w -s s
NEWGTS 'randGTS' RENAME 1 10 <% h RAND RAND NaN RAND ADDVALUE %> FOR
false
@GTStoPickledDict

top: 	b"\x80\x02}q\x00(X\n\x00\x00\x00timestampsq\x01]q\x02(I3600000000\nI7200000000\nI10800000000\nI14400000000\nI18000000000\nI21600000000\nI25200000000\nI28800000000\nI32400000000\nI36000000000\neX\x0b\x00\x00\x00randGTS.latq\x03]q\x04(G?\xedQ\xf9q\x80\x00\x00G?\xc8\x88\x7fv\x00\x00\x00G?\xc9\x1e\xa3\x16\x00\x00\x00G?\xe5\xa2y\xa1\x00\x00\x00G?\xe7\x1ft/\x80\x00\x00G?\xe2\xf3\x8b\xde\x80\x00\x00G?\xdb\xd4\xa9\xcf\x00\x00\x00G?\xe5{\x04\x02\x00\x00\x00G?\xeb\\\x8b\xde\x00\x00\x00G?\xda\xee\xe0\x16\x00\x00\x00eX\x0b\x00\x00\x00randGTS.lonq\x05]q\x06(G?\xe37\xb2l\x00\x00\x00G?\xd3Bn\xf0\x00\x00\x00G?\xdf\x05*\xfc\x00\x00\x00G?\xd9@p\x18\x00\x00\x00G?\xd6\x06\x0b$\x00\x00\x00G?\xe2\x03\n\x1d\x00\x00\x00G?\xd3\xa9\xfeD\x00\x00\x00G?\xe2\xf4\x90\xd1\x00\x00\x00G?\xd7A&\xea\x00\x00\x00G?\xb3\xcb\xa5\xa8\x00\x00\x00eX\x07\x00\x00\x00randGTSq\x07]q\x08(G?\xe1\xe3\xe9`/\xde5G?\xd6'\xa7u`\xd9\xe6G?\xbcf\xbf\xc3\xa7Q8G?\xe8v\x92\x82\x1b\x9f,G?\xdf\x99X\xaf\xd7O\x18G?\xda6Ac\x8f\x92\xc2G?\xee:\x

In [6]:
%%w -s s
NEWGTS 'randGTS' RENAME 1 10 <% h RAND RAND NaN RAND ADDVALUE %> FOR
{ 'key' 'value' 'k' 'v' } RELABEL
true
@GTStoPickledDict

top: 	b"\x80\x02}q\x00(X\n\x00\x00\x00timestampsq\x01]q\x02(I3600000000\nI7200000000\nI10800000000\nI14400000000\nI18000000000\nI21600000000\nI25200000000\nI28800000000\nI32400000000\nI36000000000\neX\x1a\x00\x00\x00randGTS{k=v,key=value}.latq\x03]q\x04(G?\xe50\x1b\xf1\x80\x00\x00G?\xe5\xe8\x05u\x00\x00\x00G?\xd8;\x07\x1e\x00\x00\x00G?\xee\xb4X\xd4\x00\x00\x00G?\xee\x80\xaf\xc0\x00\x00\x00G?\xe1V\x05\xb8\x80\x00\x00G?\xd1\xf04\x15\x00\x00\x00G?\xd5\xf0e\xa3\x00\x00\x00G?\xd8\xa2\x89p\x00\x00\x00G?\xec\xb6 \xd0\x00\x00\x00eX\x1a\x00\x00\x00randGTS{k=v,key=value}.lonq\x05]q\x06(G?\xe8\xd5\x84$\x00\x00\x00G?\xc5\xc3\x93\x84\x00\x00\x00G?\xe2\xa8}|\x00\x00\x00G?\x95\x12\x11@\x00\x00\x00G?\xed\xc5C4\x00\x00\x00G?\x90\xea\x9c\xe0\x00\x00\x00G?\xd8\xf7\xe8R\x00\x00\x00G?\xe9\xbeo\xc7\x00\x00\x00G?\xd5|\x17H\x00\x00\x00G?\xcb\xf7\x9b$\x00\x00\x00eX\x16\x00\x00\x00randGTS{k=v,key=value}q\x07]q\x08(G?\xe4{*\xce\x12\xff\xeaG?\x8e\xd1B>ph\x80G?\xd7\r\xab\xa1]\xfc\xcaG?\xcc\\\xd1\x81\xae'PG?\xdc\xd

In [7]:
%%w -s s
NEWGTS 'nogeoTS' RENAME 2 11 <% h NaN NaN NaN RAND ADDVALUE %> FOR
false
@GTStoPickledDict

top: 	b'\x80\x02}q\x00(X\n\x00\x00\x00timestampsq\x01]q\x02(I7200000000\nI10800000000\nI14400000000\nI18000000000\nI21600000000\nI25200000000\nI28800000000\nI32400000000\nI36000000000\nI39600000000\neX\x07\x00\x00\x00nogeoTSq\x03]q\x04(G?\xe1U\xf5/\xef\xbd\x82G?\xe7\xf7I\x14\xdb\x9b\xc0G?\xed\xc6\x85\xba\xce\x16\xf5G?\xbc\x9d\xc2V\xc8.hG?v\xebZ;\xc1H\x00G?\xd1\xb1\xdfB{\x16pG?\xcfGu\xbd\xf1\x8a,G?\xd6uR+P\xbd\xa8G?\xe5\xa5(\x9b\x9e\xc2\xafG?\x94\xd0;=\x18L\x00eu.'
1: 	b"\x80\x02}q\x00(X\n\x00\x00\x00timestampsq\x01]q\x02(I3600000000\nI7200000000\nI10800000000\nI14400000000\nI18000000000\nI21600000000\nI25200000000\nI28800000000\nI32400000000\nI36000000000\neX\x1a\x00\x00\x00randGTS{k=v,key=value}.latq\x03]q\x04(G?\xe50\x1b\xf1\x80\x00\x00G?\xe5\xe8\x05u\x00\x00\x00G?\xd8;\x07\x1e\x00\x00\x00G?\xee\xb4X\xd4\x00\x00\x00G?\xee\x80\xaf\xc0\x00\x00\x00G?\xe1V\x05\xb8\x80\x00\x00G?\xd1\xf04\x15\x00\x00\x00G?\xd5\xf0e\xa3\x00\x00\x00G?\xd8\xa2\x89p\x00\x00\x00G?\xec\xb6 \xd0\x00\x00\x00eX\x1a

In [8]:
top  = s.pop()
second = s.pop()
third = s.pop()

In [9]:
pd.DataFrame.from_dict(pkl.loads(top))

,timestamps,nogeoTS
0,7200000000,0.541743
1,10800000000,0.748936
2,14400000000,0.930484
3,18000000000,0.111782
4,21600000000,0.005596
5,25200000000,0.276481
6,28800000000,0.244368
7,32400000000,0.350911
8,36000000000,0.676411
9,39600000000,0.020326


In [10]:
pd.DataFrame.from_dict(pkl.loads(second))

,timestamps,"randGTS{k=v,key=value}.lat","randGTS{k=v,key=value}.lon","randGTS{k=v,key=value}"
0,3600000000,0.662123,0.776064,0.640035
1,7200000000,0.684573,0.170031,0.015048
2,10800000000,0.378603,0.583068,0.360209
3,14400000000,0.959515,0.020577,0.221583
4,18000000000,0.953209,0.930330,0.450334
5,21600000000,0.541751,0.016520,0.530493
6,25200000000,0.280286,0.390131,0.370876
7,28800000000,0.342798,0.804497,0.597459
8,32400000000,0.384920,0.335699,0.141782
9,36000000000,0.897232,0.218494,0.299991


In [11]:
pd.DataFrame.from_dict(pkl.loads(third))

,timestamps,randGTS.lat,randGTS.lon,randGTS
0,3600000000,0.916257,0.600549,0.559071
1,7200000000,0.191666,0.300930,0.346170
2,10800000000,0.196247,0.484690,0.110943
3,14400000000,0.676083,0.394558,0.764474
4,18000000000,0.722590,0.344119,0.493735
5,21600000000,0.592230,0.562871,0.409561
6,25200000000,0.434855,0.307251,0.944659
7,28800000000,0.671267,0.592354,0.813923
8,32400000000,0.855047,0.363352,0.116957
9,36000000000,0.420830,0.077326,0.128202


In [12]:
s.push([])

In [13]:
s

top: 	[]

In [14]:
%%w -s s
TYPEOF

top: 	'LIST'



In [15]:
pp = pd.DataFrame.from_dict(pkl.loads(third)).to_dict('list')

In [16]:
pp

{'timestamps': [3600000000,
  7200000000,
  10800000000,
  14400000000,
  18000000000,
  21600000000,
  25200000000,
  28800000000,
  32400000000,
  36000000000],
 'randGTS.lat': [0.916256639175117,
  0.19166558515280485,
  0.1962474687024951,
  0.6760833878070116,
  0.7225895812734962,
  0.5922297807410359,
  0.4348549386486411,
  0.6712665595114231,
  0.8550471626222134,
  0.4208297934383154],
 'randGTS.lon': [0.6005489453673363,
  0.3009297698736191,
  0.48469042405486107,
  0.39455797523260117,
  0.3441188670694828,
  0.5628710333257914,
  0.30725056305527687,
  0.5923542100936174,
  0.3633515629917383,
  0.07732615806162357],
 'randGTS': [0.5590712431575812,
  0.3461702963135437,
  0.11094282652567677,
  0.7644741574449418,
  0.49373452350595093,
  0.409561488368393,
  0.9446589570006518,
  0.8139230986905774,
  0.11695742692202649,
  0.12820231948088257]}

In [17]:
s.push(pp)

In [18]:
s

top: 	{'randGTS.lat': [0.916256639175117, 0.19166558515280485, 0.1962474687024951, 0.6760833878070116, 0.7225895812734962, 0.5922297807410359, 0.4348549386486411, 0.6712665595114231, 0.8550471626222134, 0.4208297934383154], 'timestamps': [3600000000, 7200000000, 10800000000, 14400000000, 18000000000, 21600000000, 25200000000, 28800000000, 32400000000, 36000000000], 'randGTS': [0.5590712431575812, 0.3461702963135437, 0.11094282652567677, 0.7644741574449418, 0.49373452350595093, 0.409561488368393, 0.9446589570006518, 0.8139230986905774, 0.11695742692202649, 0.12820231948088257], 'randGTS.lon': [0.6005489453673363, 0.3009297698736191, 0.48469042405486107, 0.39455797523260117, 0.3441188670694828, 0.5628710333257914, 0.30725056305527687, 0.5923542100936174, 0.3633515629917383, 0.07732615806162357]}
1: 	'LIST'